In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("Lab_4_data_fitting.ipynb")

# Lab week 4: Data fitting to see if we can detect a grasp success y/n

Can you use the finger 1 stop time and max value to tell the difference between a successful and a failed grasp?

Lab slides: https://docs.google.com/presentation/d/1NtoXYbl2nq1dkIU0KQE8ogjSCfb22wgLigY3JJpHHGI/edit?usp=sharing


In [ ]:
# Install a pip package in the current Jupyter kernel
# Note: this is a safety check in case your JN install was not in the same place as your libraries
import sys
!{sys.executable} -m pip install scipy

In [ ]:
# Doing the imports for you
import numpy as np
import json as json
from scipy.stats import linregress
import matplotlib.pyplot as plt
from numpy.polynomial import polynomial

## Read the data in 

In [ ]:
# TODO: Read the same data in as for the lecture activity and put it in the data_motor_position variable and make
#   your ts values

time_step = 1/30


data_motor_position = ...  # Motor position f1.csv
ts = ...

# 

In [ ]:
grader.check("Read data")

## Box plot of fitted lines for all data

In the lecture activity you fit a line to just one row (each) of the successful versus failed grasps. In this lab you'll fit the line to ALL of the grasps. For each grasp, save the end point (time at the **y_max** value and the **y_max** value and the error of the fit). You'll be making a box plot of this data to see if either look different for successful versus failed grasps.

Why a box plot? Box plots are better for visually showing multiple distributions in the same plot, where they can be compared side-by-side.

See lab slides for what this should look like when you're done.

In [ ]:
# TODO: Copy over fit_line_to_middle from the lecture activity

In [ ]:
# TEST CELL
# TODO: Check that it works


In [ ]:
# EXAMPLE CODE
# This bit of code shows how to do the nx2 array part

# Make the nx2 array - one row for each element
np_array_for_res = np.zeros((data_motor_position.shape[0], 2))
# For each row
for row in range(0, data_motor_position.shape[0]):
    # Put the min value in the first column, the max in the second
    np_array_for_res[row, 0] = np.min(data_motor_position[row, :])    
    np_array_for_res[row, 1] = np.max(data_motor_position[row, :])

    # This also works
    min_value = np.min(data_motor_position[row, :]) 
    max_value = np.max(data_motor_position[row, :])
    np_array_for_res[row, :] = [min_value, max_value]

print(np_array_for_res)

In [ ]:
def fit_middle_all(ts, data):
    """ Fit the middle for every row of data
    @param ts - the t values to use
    @param data - n picks X timesteps
    @returns all end times, max values, and errors as one numpy array (n rows X 3)"""


    # TODO:
    #.    make numpy array to put data in (nx3)
    #.    for each row in data
    #.       start_pt, end_pt, err_fit = fit_line_to_middle_bit(ts, current row)
    #        copy end_pt and err_fit information into appropriate row in nx3 output
    #. return nx3 numpy array
    ...

In [ ]:
# TEST CELL
# Write the code to call the function here

# The successful rows go from 0 to 354
last_row_successful = 355

# The unsuccessful rows are the remaining ones


# TODO:
#. Call your function twice, once with the successful data, once with the failed
fit_data_successful = ...
fit_data_failed = ...

# Note for both of these: Small changes in how you index can result in slightly different numbers,
#  but the overall magnitude should be correct

# Note: Doing np.mean(nx3 array, 0) tells numpy to do the mean over each column (instead of the mean
#  over the entire nx3 array). So it should return 3 numbers - the mean of the times, the mean
#  of the max y values, and the mean of the fit error

# Should print the following, although the actual decimal places may differ
#  Mean successful [3.98791579e-01 4.07402569e+02 4.21582062e+00] and failed [  0.40810473 392.76044707   5.51566179]
print(f"Mean successful {np.mean(fit_data_successful, 0)} and failed {np.mean(fit_data_failed, 0)}")

In [ ]:
grader.check("Fitted_line_code")

<!-- BEGIN QUESTION -->

## Do the box plots

- Left window: t values of y_max point (the first column of fit_data_*)

- Middle window: y_max values (the second column of fit_data_*)

- Right window: fit error values (the third column of fit_data_*)

Note that boxplot uses labels for labeling the x axis of the box plot.

Implementation order: for plotting, try calling box plot with just one column of data. Then add the second. 

In [ ]:
# Three plots
n_rows = 1
n_cols = 3
fig, axs = plt.subplots(n_rows, n_cols, figsize=(8, 3))

...

# Do the box plots...

In [ ]:
# Manual grade
print("This is a manually-graded question; there is no grader.check() function. See rubric and slides for more information on expected output.")

<!-- END QUESTION -->

# Part 2: Fitting a curve to the wrist force data

## First step: Getting the data

Along with the motor data we also have force data at the wrist. The wrist force data has three channels (x,y,z). For this assignment, we want to get out the total magnitude of the force at the wrist, rather than the force in each direction individually:

Equation: **sqrt (sum x,y,z (wrist force data)^2)**

The data is stored in **Wrist_force_data.csv** in the Data directory. 

In [ ]:

# TODO: Read in the data; should be 660 x 120 (x,y,z data for 40 time steps)
data_wrist_force_xyz = ...


In [ ]:
# TODO: Calculate the total force (see equation above)

# TODO: Calculate the magnitude of the force
data_wrist_force_magnitude = ...

assert data_wrist_force_magnitude.shape == (660, 40)
assert np.isclose(data_wrist_force_magnitude[0, 0], 6.5389367, atol=0.001)
assert np.isclose(np.max(data_wrist_force_magnitude), 20.46059305836466, atol=0.001)

In [ ]:
# Plot a few of the rows to see what the data looks like
n_rows = 1
n_cols = 4
fig, axs = plt.subplots(n_rows, n_cols, figsize=(8, 2))


# TODO Pick 4 examples to plot. Use your ts values from above. You do not need to
#   use the ones from the slides, but you should but the example number in the title
# Don't forget to label the x and y axes
plt.tight_layout()

In [ ]:
grader.check("Fit_curve")

## Write a function to fit a polynomial to one row of the data

Since we aren't sure what a "good" polynomial fit might be, make that a parameter

In [ ]:
def fit_poly(ts, data, deg):
    """ Fit a polynomial to the data
    @param ts - the x values
    @param data - the y values
    @param degree of the polynomial. Should be an integer
    @return a list of polynomial coefficients (see polynomial.polyfit)"""
    ... # Call polynomial.polyfit


In [ ]:
# Test

check_poly = fit_poly(ts, data_wrist_force_magnitude[10], deg=2)


assert np.all(np.isclose(check_poly, [10.0970104, -13.91458045, 10.73089215], atol=0.01))

In [ ]:
grader.check("fit_cubic")

<!-- BEGIN QUESTION -->

## Check the fit

It's not always clear what a "good" fit is. In this part of the code we'll plot a rset of examples, with increasing degrees of polynomials.

Sometimes you just want to plot some examples but don't really care which ones; this problem also has a solution to the "I have space for n plots, I want to skip over some of the data" problem

In [ ]:
# EXAMPLE CODE - row column indexing, how to put n things into a grid

# If you have n things, and you want to put them in r rows, first figure out how many columns you'll need
#  Integer division: 
#    n // r  gives the number of times r can go into n
#    n % r gives the left over - it is always in the range 0 to n-1
n = 15
r = 4
c = n // r  # this gives the floor of n / r, which in this case will be 15/4 is 3
if n % r != 0:  # % is modulus - unless r goes exactly into n we need to add a bit (some of the last columns will be empty)
    c += 1   # Make c one bigger

# Go through all of n
for i in range(0, n):
    row = i // r   # Which row we're on
    col = i % r    # Which column we're on
    end_str = " "       # Use a space at the end of the pring
    if col == r-1:
        end_str = "\n"  # ... except at the end of the row
    print(f"n{i}r{row}c{col} ", end=end_str)  # No return


In [ ]:
# EXAMPLE CODE - zip
# These two pieces of code do the same thing
a_str = ("foo", "bar")
b_number = (3, 10)
for i in range(len(a_str)):
    a = a_str[i]
    b = b_number[i]
    print(f"Loop iteration {i} has {a} and {b}")

# Look, no i!
for a, b in zip(a_str, b_number):
    print(f"Loop iteration... um... has {a} and {b}")

In [ ]:
# SCRATCH CELL
# Practice the plot for one row of data

In [ ]:
# Note: This code will all work even if you change the number of rows/columns
n_rows = 4
n_cols = 6
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 3, n_rows * 3))

n_total_plots = n_rows * n_cols
# take every nth
n_skip = data_wrist_force_magnitude.shape[0] // n_total_plots

color_deg = ("maroon", "steelblue", "olivedrab")
deg_fit = (1, 3, 6)
lstyle = ('dashed', 'dotted', 'dashdot')
# This loops over the total number of plots we can fit
for i in range(0, n_total_plots):
    # Yes, this works - it's the same as doing r = row_offset + p // ncols on one line,
    #   and c = p % n_cols on the second line
    r, c = i // n_cols, i % n_cols
    data_row_indx = i * n_skip

    # Plot 4 things in this figure:
    #   the original data (in black)
    #   polynomial fits of degree 1, 3, 6
    # Note: use axs[r, c]
    # Note 2: The data row you want is in data_row_indx...
    # Note 3: Use polynomial.polyval(ts, ret_poly) to create y values for your t values
    ...

plt.tight_layout()

In [ ]:
# Manual grade
print("This is a manually-graded question; there is no grader.check() function. See rubric and slides for more information on expected output.")

<!-- END QUESTION -->

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:

# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS I25 (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

In [ ]:
grader.check("hours_collaborators")

### To submit

- Do a restart then run all to make sure everything runs ok and the plots are showing
- Save the file
- Submit just this .ipynb file through gradescope, Lab 4, fitting
- You do NOT need to submit the data files - we will supply those
- Where there are given variable/file names (eg, foo = ...) DON'T change those, or the autograder will fail

If the Gradescope autograder fails, please check here first for common reasons for it to fail
    https://docs.google.com/presentation/d/1tYa5oycUiG4YhXUq5vHvPOpWJ4k_xUPp2rUNIL7Q9RI/edit?usp=sharing

Most likely failure for this assignment is not naming the data directory and files correctly; capitalization matters for the Gradescope grader. 

A gentle reminder that you should NOT be copying and pasting code (let alone sending files) to other people in the class.